In [ ]:
# Execute this cell to install dependencies
%pip install sf-hamilton[visualization]

# Conttribution: Translate to Hamilton [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dagworks-inc/hamilton/blob/main/examples/contrib/notebooks/dagworks-translate_to_hamilton.ipynb) [![GitHub badge](https://img.shields.io/badge/github-view_source-2b3137?logo=github)](https://github.com/dagworks-inc/hamilton/blob/main/examples/contrib/notebooks/dagworks-translate_to_hamilton.ipynb)


In [ ]:
from hamilton import driver, dataflows

In [ ]:
# import module -- pick one
# translate_to_hamilton = dataflows.import_module('translate_to_hamilton')
# from hamilton.contrib.dagworks import translate_to_hamilton

In [6]:
# find some code to change.
def my_func(b, c):
    """Pretend this is a big function and we want to conver the body of it to hamilton."""
    a = b + c
    d = some_func(a)
    e = d + 1
    return e

In [7]:
# get the code
import inspect
user_code = inspect.getsource(my_func)

In [8]:
user_code

'def my_func(b, c):\n    """Pretend this is a big function."""\n    a = b + c\n    d = some_func(a)\n    e = d + 1\n    return e\n'

In [9]:
# create a driver
dr = (
    driver.Builder()
        .with_config({})
        .with_modules(translate_to_hamilton)
        .build()
)

Note: Hamilton collects completely anonymous data about usage. This will help us improve Hamilton over time. See https://github.com/dagworks-inc/hamilton#usage-analytics--data-privacy for details.


In [10]:
result = dr.execute(
    ["code_segments", "translated_code_response"], # request these as outputs
    inputs={"user_code": user_code, "model_name": "gpt-4-1106-preview"}
)

In [14]:
for code in result["code_segments"]:
    print(code)

# functions.py
def a(b: float, c: float) -> float:
    """Computes the sum of b and c."""
    return b + c

def d(a: float) -> float:
    """Applies some_func to a."""
    return some_func(a)

def e(d: float) -> float:
    """Increments d by 1."""
    return d + 1

# run.py
from hamilton import driver
import functions

# Instantiate a Driver
dr = driver.Driver(config={}, module=functions)

# Specify the final output we want from the Driver
output = ["e"]

# Suppose we know the values of `b` and `c` we want to use
inputs = {"b": 1, "c": 2}

# Execute the data flow to produce the output
result = dr.execute(output, inputs=inputs)

print(result['e'])  # The result is a dictionary with keys as the output names we specified earlier


In [12]:
print(result["translated_code_response"])

In Hamilton, each part of the procedural function `my_func` that assigns a value to a variable can be refactored into its own Hamilton function. Here's how you might represent this code with separate functions:

```python
# functions.py
def a(b: float, c: float) -> float:
    """Computes the sum of b and c."""
    return b + c

def d(a: float) -> float:
    """Applies some_func to a."""
    return some_func(a)

def e(d: float) -> float:
    """Increments d by 1."""
    return d + 1
```

In this example, `some_func` is assumed to be an existing function that takes a single argument and returns a single result. In actual implementation, `some_func` would be defined elsewhere, or you would need to create a Hamilton function that correctly replicates its functionality.

Now, you'd want to set up the driver script (`run.py`) to execute these functions:

```python
# run.py
from hamilton import driver
import functions

# Instantiate a Driver
dr = driver.Driver(config={}, module=functions)

# 